In [1]:
import pandas as pd

In [189]:
df = pd.read_csv("./cmd/addr.txt",header=None,sep='\t',dtype=str,names=['type','code','name'])

def nations_():
    for line in open("./cmd/chinese_nations.txt"):
        line = line.strip()
        if len(line) > 2:
            yield line[:-1]
        yield line

nations = list(nations_())

In [8]:
province = df[df['type']=='province']

In [21]:
def process(row):
    name = row['name']
    if name.endswith('市'):
        return [name, name.rstrip('市')]
    elif name.endswith('省'):
        return [name, name.rstrip('省')]
    elif name == '内蒙古自治区':
        return [name, '内蒙古省', '内蒙古', '内蒙']
    elif name == '广西壮族自治区':
        return [name, '广西自治区', '广西省', '广西']
    elif name == '西藏自治区':
        return [name, '西藏省', '西藏']
    elif name == '新疆维吾尔自治区':
        return [name,'新疆维吾尔族自治区', '新疆自治区', '新疆省', '新疆']
    return [name]

In [26]:
#province['ext'] = province.apply(process, axis=1)

In [27]:
city = df[df['type']=='city']

In [190]:
def _zhou(name):
    x = name.rstrip('自治州')
    for i in nations:
        x = x.replace(i,"")
    ret = [name,x+"自治州",x+"州",x+"市",x]
    if name == '克孜勒苏柯尔克孜自治州': ret.append("克州")
    elif name == '巴音郭楞蒙古自治州': ret.append("巴州")
    elif name == '博尔塔拉蒙古自治州': ret.append("博州")
    return ret
def city_process(row):
    name = row['name']
    if name.endswith('市'):
        ret = [name, name.rstrip('市')]
        if name == '呼和浩特市': ret.append('呼市')
        return ret
    elif name.endswith('盟'):
        return [name, name.rstrip('盟')]
    elif name.endswith("自治州"):
        return _zhou(name)
    elif name.endswith("地区"):
        return [name, name.rstrip("地区")]
    return [name]

In [192]:
#city[~city.name.str.match('.*(市|盟|区|区划|县)$')].name.apply(_zhou)
#city.apply(city_process,axis=1)
#city[~city.name.str.match('.*(市|盟|自治州|地区)$')]

In [87]:
county = df[df['type']=='county']

In [256]:
# TODO
county_suffix = ['新区', '矿区','区', '县', '市', '现代产业园', '行政委员会', '管委会']
cnames = set(['市辖区'])
def county_process(row):
    name = row['name']
    if len(name) <= 2 or name in cnames:
        return [name] # 赵县
    s = '高新技术产业开发区'
    if name.endswith(s):
        x = name[:-len(s)].rstrip('市')
        if len(x) <= 3 or x == '乌鲁木齐':
            return [name, s, '高新区']
        if x == '河北衡水':
            return [name, s, '高新区']
        if x == '包头稀土':
            return [name, '稀土'+s, '稀土高新区']
        if x == '长春净月':
            return [name, '净月'+s, '净月高新区', '净月']
        if x == '泰州医药':
            return [name, '医药'+s, '医药高新区']
        if x == '合肥新站':
            return [name, '新站'+s, '新站高新区']
        return [name]

    s = '高新技术产业园区'
    if name.endswith(s):
        return [name, s, '高新区']

    s = '工业园区'
    if name.endswith(s):
        if name == '呼和浩特金海工业园区':
            return [name, '金海工业园区', '金海', s, '工业园']
        if name =='格尔木藏青工业园区':
            return [name, '藏青工业园区', '藏青', s, '工业园']
        return [name, s, '工业园']
    
    if name == '石家庄循环化工园区':
        return [name, '循环化工园', '化工园']
    
    ss = ['经济技术开发区','经济开发区']
    for s in ss:
        if name.endswith(s):
            return [name, s, '开发区']
    
    if name.endswith('自治县'):
        x = name[:-3]
        for i in nations:
            x = x.replace(i,"")
        if len(x) == 0: return [name]
        ret = [name,x+"自治县",x+"县",x]
        return ret    
    for suffix in county_suffix:
        if name.endswith(suffix):
            return [name, name[:-len(suffix)]]
    if name.endswith('自治旗'):
        x = name[:-len('自治旗')]
        return [name, x+'旗', x]
    elif name.endswith('旗'):
        return [name, name.rstrip('旗')]
    elif name == '保定白沟新城':
        return [name, '白沟']
    elif name.endswith('群岛'):
        return [name, name.rstrip('群岛')]
    elif name == '中沙群岛的岛礁及其海域':
        return [name, '中沙群岛', '中沙']
    return [name]

In [257]:
#county.apply(county_process,axis=1)
#county[~county.name.str.match('.*(县|区|市|现代产业园|行政委员会|管委会|自治旗|旗|保定白沟新城|群岛)$')]
county[county.name.str.match('.*(管理)(区)$')].apply(county_process,axis=1)
#county[county.name.str.match('.*(自治县)$')].apply(county_process,axis=1)

444       [唐山市汉沽管理区, 唐山市汉沽管理]
547     [张家口市察北管理区, 张家口市察北管理]
548     [张家口市塞北管理区, 张家口市塞北管理]
2237          [龙感湖管理区, 龙感湖管理]
2330      [岳阳市屈原管理区, 岳阳市屈原管理]
2342    [常德市西洞庭管理区, 常德市西洞庭管理]
2355    [益阳市大通湖管理区, 益阳市大通湖管理]
2383      [永州市金洞管理区, 永州市金洞管理]
2384    [永州市回龙圩管理区, 永州市回龙圩管理]
2397      [怀化市洪江管理区, 怀化市洪江管理]
dtype: object

In [112]:
town = df[df['type']=='town']

In [116]:
town.name.to_csv('town.txt',index=None,header=None)

In [117]:
!goname -i town.txt

 43486 / 43486 [====================================================] 100.00% 0s
done!


In [135]:
town_suffix = ['镇', '乡', '街道办事处', '街道', '地区办事处']
def town_process(row):
    name = row['name']
    for suffix in town_suffix:
        if name.endswith(suffix):
            return [name, name[:-len(suffix)]]
    return [name]

In [139]:
#town.apply(town_process,axis=1)
#town[~town.name.str.match('.*(镇|乡|街道办事处|地区办事处|街道)$')]

In [142]:
village = df[df['type']=='village']

In [143]:
village.name.to_csv('village.txt',index=None,header=None)

In [144]:
!goname -i village.txt -o village_suffix.txt

 270546 / 270546 [==================================================] 100.00% 8s
done!


In [174]:
def village_process(row):
    name = row['name']
    ss = ["村委会","村民委员会","社区村委会","嘎查","嘎查村","嘎查村委员会"]
    for s in ss:
        if name.endswith(s):
            x = name[:-len(s)]
            if x.endswith('村'):
                return [name, x, x.rstrip('村')]
            else:
                return [name, x+'村', x]
    
    ss = ["社区居委会","社区居民委员会","居民委员会","居委会","社区","社区委员会","社区居委","社区居民委会"]
    for s in ss:
        if name.endswith(s):
            x = name[:-len(s)]
            return [name, x+'社区', x+"小区", x]

    ss = ["生活区","工作区","工业园","工业区"]
    for s in ss:
        if name.endswith(s):
            x = name[:-len(s)]
            return [name, x]
    return [name]

In [178]:
#village[village.name.str.match(".*(嘎查)$")].apply(village_process, axis=1)
ss = ["村委会","村民委员会","社区村委会","嘎查","嘎查村","嘎查村委员会"]+["社区居委会","社区居民委员会"]+["社区"]
ss += ["居民委员会","居委会","社区","社区委员会","社区居委","生活区","工作区","工业园","工业区","社区居民委会"]
village[~village.name.str.match(".*("+'|'.join(ss)+")$")].head()

,type,code,name
52121,village,110115004402,亦庄镇南部多功能配套区
52236,village,110115006402,南郊旧宫场区
52439,village,110115103401,青云店镇工业开发区
52498,village,110115104401,北京采育经济开发区
52695,village,110115108401,庞各庄镇开发区


In [201]:
nations.sort(reverse=True)

In [202]:
nations

['黎族',
 '高山族',
 '高山',
 '阿昌族',
 '阿昌',
 '门巴族',
 '门巴',
 '锡伯族',
 '锡伯',
 '鄂温克族',
 '鄂温克',
 '鄂伦春族',
 '鄂伦春',
 '达斡尔族',
 '达斡尔',
 '赫哲族',
 '赫哲',
 '裕固族',
 '裕固',
 '藏族',
 '蒙古族',
 '蒙古',
 '苗族',
 '羌族',
 '维吾尔族',
 '维吾尔',
 '纳西族',
 '纳西',
 '白族',
 '畲族',
 '瑶族',
 '珞巴族',
 '珞巴',
 '独龙族',
 '独龙',
 '满族',
 '汉族',
 '水族',
 '毛南族',
 '毛南',
 '柯尔克孜族',
 '柯尔克孜',
 '朝鲜族',
 '朝鲜',
 '景颇族',
 '景颇',
 '普米族',
 '普米',
 '撒拉族',
 '撒拉',
 '拉祜族',
 '拉祜',
 '怒族',
 '德昂族',
 '德昂',
 '彝族',
 '布朗族',
 '布朗',
 '布依族',
 '布依',
 '壮族',
 '塔塔尔族',
 '塔塔尔',
 '塔吉克族',
 '塔吉克',
 '基诺族',
 '基诺',
 '土族',
 '土家族',
 '土家',
 '回族',
 '哈萨克族',
 '哈萨克',
 '哈尼族',
 '哈尼',
 '傣族',
 '傈僳族',
 '傈僳',
 '保安族',
 '保安',
 '俄罗斯族',
 '俄罗斯',
 '侗族',
 '佤族',
 '仫佬族',
 '仫佬',
 '仡佬族',
 '仡佬',
 '京族',
 '乌孜别克族',
 '乌孜别克',
 '东乡族',
 '东乡']